# **Métodos de Detecção de Outliers**

# 1. Imports

In [1]:
import numpy as np
import yfinance as yf
from utils import adf_test
from detect_outliers import statistical_method, isolation_forest_method
from plots import time_series_plot

# 2. Dados

In [2]:
# Carregar os dados
df = yf.download("^NDX", period="2y", interval="1h")[["Adj Close"]]

# Resetar o índice para ter a coluna de data separada
df.reset_index(inplace=True)

[*********************100%%**********************]  1 of 1 completed


In [3]:
time_series_plot(
    df=df,
    target_col="Adj Close",
    title="Nasdaq-100 Adjusted Close Price Over 2 Years (Hourly)",
    width=1500,
    height=600
)


### 2.1. Calculando o logaritmo dos retornos

In [4]:
df.loc[:, 'log_return'] = np.log(df.loc[:, 'Adj Close'] / df.loc[:, 'Adj Close'].shift(1))
df = df.dropna()

/tmp/ipykernel_53609/247422845.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [5]:
# Plotar os dados usando Plotly
time_series_plot(
    df=df,
    target_col="log_return",
    title="Nasdaq-100 log_return Over 2 Years (Hourly)",
)

### 2.2. Teste de estacionariedade

In [6]:
adf_test(df["Adj Close"])
adf_test(df["log_return"])

p-value (Adj Close): 0.961483
Não rejeitamos a hipótese nula de que a série Adj Close tem uma raiz unitária. Portanto, a série Adj Close não é estacionária.
p-value (log_return): 0.000000
Rejeitamos a hipótese nula de que a série log_return tem uma raiz unitária. Portanto, a série log_return é estacionária.


# 3. Detecção de Outliers - Método 1 - Estatístico

In [7]:
statistical_method(df=df, target_col="log_return", threshold=3)

In [8]:
# px.histogram(df["log_return"])

Problemas desse método:
- Outliers extremos podem inflar o desvio padrão, tornando mais difícil a detecção de outros outliers.
- Não considera dependência temporal
- O método não se adapta a mudanças na volatilidade ao longo do tempo. Em períodos de alta volatilidade, mais pontos podem ser classificados erroneamente como outliers, enquanto em períodos de baixa volatilidade, menos outliers podem ser detectado
  
Ou seja, é um método muito simples para dados mais complexos, como séries temporais financeiras, porém podem ser úteis em tratamento de outliers em problemas supervisionados mais tradicionais, em especial para dados mais simples e estruturados.

# 4. Detecção de Outliers - Método 2 - Isolation Forest

In [9]:
isolation_forest_method(
    df=df, target_col="log_return", n_estimators=500, contamination=0.001
)

Prós desse método:

- Robusto a outliers extremos, por se tratar de um método de ensemble.
- Consegue detectar padrões mais complexos nos dados, em relação ao primeiro método

Contras desse método:

- Não é específico para séries temporais, apesar de apresentar resultados mais interessantes.

# 5. Detecção de Outliers - Método 3 - Autoencoder